In [ ]:
{
    "cells": [
        {
            "cell_type": "code",
            "execution_count": 1,
            "metadata": {
                "execution": {
                    "iopub.execute_input": "2021-12-09T15:42:30.890422Z",
                    "iopub.status.busy": "2021-12-09T15:42:30.890036Z",
                    "iopub.status.idle": "2021-12-09T15:42:37.774794Z",
                    "shell.execute_reply": "2021-12-09T15:42:37.774062Z",
                    "shell.execute_reply.started": "2021-12-09T15:42:30.890331Z"
                },
                "trusted": true
            },
            "outputs": [
                {
                    "name": "stderr",
                    "output_type": "stream",
                    "text": [
                        "2022-12-02 14:17:50.032968: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA\n",
                        "To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.\n",
                        "2022-12-02 14:17:51.562438: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:\n",
                        "2022-12-02 14:17:51.562735: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:\n",
                        "2022-12-02 14:17:51.562746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.\n"
                    ]
                }
            ],
            "source": [
                "from keras.preprocessing.text import Tokenizer\n",
                "from keras.utils import pad_sequences\n",
                "from nltk.tokenize import word_tokenize\n",
                "from nltk.stem import WordNetLemmatizer\n",
                "from nltk.corpus import stopwords\n",
                "from tensorflow.python.keras.layers import *\n",
                "from tensorflow.python.keras.models import Model\n",
                "import numpy as np \n",
                "import pandas as pd \n",
                "import re\n",
                "import nltk\n",
                "from preprocess import *\n",
                "from models import *"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 2,
            "metadata": {
                "execution": {
                    "iopub.execute_input": "2021-12-09T15:42:52.454527Z",
                    "iopub.status.busy": "2021-12-09T15:42:52.453714Z",
                    "iopub.status.idle": "2021-12-09T15:43:01.326923Z",
                    "shell.execute_reply": "2021-12-09T15:43:01.326227Z",
                    "shell.execute_reply.started": "2021-12-09T15:42:52.454487Z"
                },
                "trusted": true
            },
            "outputs": [
                {
                    "data": {
                        "text/html": [
                            "<div>\n",
                            "<style scoped>\n",
                            "    .dataframe tbody tr th:only-of-type {\n",
                            "        vertical-align: middle;\n",
                            "    }\n",
                            "\n",
                            "    .dataframe tbody tr th {\n",
                            "        vertical-align: top;\n",
                            "    }\n",
                            "\n",
                            "    .dataframe thead th {\n",
                            "        text-align: right;\n",
                            "    }\n",
                            "</style>\n",
                            "<table border=\"1\" class=\"dataframe\">\n",
                            "  <thead>\n",
                            "    <tr style=\"text-align: right;\">\n",
                            "      <th></th>\n",
                            "      <th>id</th>\n",
                            "      <th>qid1</th>\n",
                            "      <th>qid2</th>\n",
                            "      <th>question1</th>\n",
                            "      <th>question2</th>\n",
                            "      <th>is_duplicate</th>\n",
                            "    </tr>\n",
                            "  </thead>\n",
                            "  <tbody>\n",
                            "    <tr>\n",
                            "      <th>0</th>\n",
                            "      <td>0</td>\n",
                            "      <td>1</td>\n",
                            "      <td>2</td>\n",
                            "      <td>What is the step by step guide to invest in sh...</td>\n",
                            "      <td>What is the step by step guide to invest in sh...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>1</th>\n",
                            "      <td>1</td>\n",
                            "      <td>3</td>\n",
                            "      <td>4</td>\n",
                            "      <td>What is the story of Kohinoor (Koh-i-Noor) Dia...</td>\n",
                            "      <td>What would happen if the Indian government sto...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>2</th>\n",
                            "      <td>2</td>\n",
                            "      <td>5</td>\n",
                            "      <td>6</td>\n",
                            "      <td>How can I increase the speed of my internet co...</td>\n",
                            "      <td>How can Internet speed be increased by hacking...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>3</th>\n",
                            "      <td>3</td>\n",
                            "      <td>7</td>\n",
                            "      <td>8</td>\n",
                            "      <td>Why am I mentally very lonely? How can I solve...</td>\n",
                            "      <td>Find the remainder when [math]23^{24}[/math] i...</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "    <tr>\n",
                            "      <th>4</th>\n",
                            "      <td>4</td>\n",
                            "      <td>9</td>\n",
                            "      <td>10</td>\n",
                            "      <td>Which one dissolve in water quikly sugar, salt...</td>\n",
                            "      <td>Which fish would survive in salt water?</td>\n",
                            "      <td>0</td>\n",
                            "    </tr>\n",
                            "  </tbody>\n",
                            "</table>\n",
                            "</div>"
                        ],
                        "text/plain": [
                            "   id  qid1  qid2                                          question1  \\\n",
                            "0   0     1     2  What is the step by step guide to invest in sh...   \n",
                            "1   1     3     4  What is the story of Kohinoor (Koh-i-Noor) Dia...   \n",
                            "2   2     5     6  How can I increase the speed of my internet co...   \n",
                            "3   3     7     8  Why am I mentally very lonely? How can I solve...   \n",
                            "4   4     9    10  Which one dissolve in water quikly sugar, salt...   \n",
                            "\n",
                            "                                           question2  is_duplicate  \n",
                            "0  What is the step by step guide to invest in sh...             0  \n",
                            "1  What would happen if the Indian government sto...             0  \n",
                            "2  How can Internet speed be increased by hacking...             0  \n",
                            "3  Find the remainder when [math]23^{24}[/math] i...             0  \n",
                            "4            Which fish would survive in salt water?             0  "
                        ]
                    },
                    "execution_count": 2,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "df = pd.read_csv(\"questions.csv\")\n",
                "df.head()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 3,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "text/plain": [
                            "(404351, 6)"
                        ]
                    },
                    "execution_count": 3,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "df.shape"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 4,
            "metadata": {},
            "outputs": [
                {
                    "data": {
                        "text/plain": [
                            "(100000, 6)"
                        ]
                    },
                    "execution_count": 4,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "#pick a sample of 5000 distinct random rows\n",
                "df = df.sample(n=100000, random_state=1)\n",
                "df.shape"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 5,
            "metadata": {
                "execution": {
                    "iopub.execute_input": "2021-12-07T01:21:29.482516Z",
                    "iopub.status.busy": "2021-12-07T01:21:29.482071Z",
                    "iopub.status.idle": "2021-12-07T01:21:29.56485Z",
                    "shell.execute_reply": "2021-12-07T01:21:29.564017Z",
                    "shell.execute_reply.started": "2021-12-07T01:21:29.482481Z"
                },
                "trusted": true
            },
            "outputs": [],
            "source": [
                "# question_1, question_2 = df['question1'].to_list(), df['question2'].to_list()\n",
                "# is_duplicate = df['is_duplicate'].to_list()\n",
                "# preprocess_neural(question_1, question_2, is_duplicate)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 6,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "q1_inp, q2_inp, is_duplicate = df['question1'].to_list(), df['question2'].to_list(), df['is_duplicate'].to_list()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": [
                "Acquired Test data"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 7,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "MAX_WORDS_VOCAB = 200000\n",
                "tokenizer = Tokenizer(num_words = MAX_WORDS_VOCAB, lower=False, split=\" \")\n",
                "tokenizer.fit_on_texts(list(df['question1'].values.astype(str))+list(df['question2'].values.astype(str)))"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 8,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Number of words in vocabulary:  60949\n"
                    ]
                }
            ],
            "source": [
                "print(\"Number of words in vocabulary: \", len(tokenizer.word_index))"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 9,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "\n",
                "q1_sequence = tokenizer.texts_to_sequences(df['question1'].values.astype(str))\n",
                "q1_sequence = pad_sequences(q1_sequence, maxlen = 128)\n",
                "\n",
                "q2_sequence = tokenizer.texts_to_sequences(df['question2'].values.astype(str))\n",
                "q2_sequence = pad_sequences(q2_sequence, maxlen = 128)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 10,
            "metadata": {},
            "outputs": [],
            "source": [
                "windex = tokenizer.word_index"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 11,
            "metadata": {
                "trusted": true
            },
            "outputs": [],
            "source": [
                "embedding_index = {}\n",
                "with open('glove.6B.300d.txt','r') as f:\n",
                "    for line in f:\n",
                "        values = line.split()\n",
                "        word = values[0]\n",
                "        vectors = np.asarray(values[1:], 'float32')\n",
                "        embedding_index[word] = vectors\n",
                "    f.close()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 12,
            "metadata": {
                "trusted": true
            },
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "(60950, 300)\n"
                    ]
                }
            ],
            "source": [
                "embedding_matrix = np.random.random((len(windex)+1, 300))\n",
                "\n",
                "for word, i in windex.items():\n",
                "    embedding_vector = embedding_index.get(word)\n",
                "    if embedding_vector is not None:\n",
                "        embedding_matrix[i] = embedding_vector\n",
                "\n",
                "print(embedding_matrix.shape)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 13,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "60950\n"
                    ]
                }
            ],
            "source": [
                "print(len(windex)+1)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 14,
            "metadata": {},
            "outputs": [],
            "source": [
                "#split the data into 70-20-10 train-validation-test with random state 42\n",
                "from sklearn.model_selection import train_test_split\n",
                "q1_train, q1_test, q2_train, q2_test, y_train, y_test = train_test_split(q1_sequence, q2_sequence, is_duplicate, test_size=0.1, random_state=42)\n",
                "q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(q1_train, q2_train, y_train, test_size=0.2, random_state=42)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 15,
            "metadata": {},
            "outputs": [],
            "source": [
                "import tensorflow as tf\n",
                "y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)\n",
                "y_val = tf.keras.utils.to_categorical(y_val, num_classes=2)\n",
                "y_test = tf.keras.utils.to_categorical(y_test, num_classes=2)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 16,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "(100000,)\n"
                    ]
                }
            ],
            "source": [
                "is_duplicate = np.array(is_duplicate)\n",
                "print(is_duplicate.shape)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 17,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Train:  [0.63294444 0.36705556]\n",
                        "Validation:  [0.6358333  0.36416668]\n",
                        "Test:  [0.6291 0.3709]\n"
                    ]
                }
            ],
            "source": [
                "#print the ratio of positive and negative samples in train, validation and test\n",
                "y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)\n",
                "print(\"Train: \", sum(y_train)/len(y_train))\n",
                "print(\"Validation: \", sum(y_val)/len(y_val))\n",
                "print(\"Test: \", sum(y_test)/len(y_test))"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 19,
            "metadata": {},
            "outputs": [
                {
                    "name": "stderr",
                    "output_type": "stream",
                    "text": [
                        "2022-12-02 14:18:17.626672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected\n",
                        "2022-12-02 14:18:17.626735: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist\n",
                        "2022-12-02 14:18:17.627957: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA\n",
                        "To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.\n"
                    ]
                }
            ],
            "source": [
                "model = LsTM(emb_mat = embedding_matrix, vocab_size = len(windex)+1)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 20,
            "metadata": {},
            "outputs": [],
            "source": [
                "model.train_model()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 21,
            "metadata": {},
            "outputs": [],
            "source": [
                "model = LsTM(embedding_matrix, len(windex) + 1, loss=\"categorical_crossentropy\")"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 22,
            "metadata": {},
            "outputs": [],
            "source": [
                "model.train_model()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 23,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Model: \"model_1\"\n",
                        "__________________________________________________________________________________________________\n",
                        " Layer (type)                   Output Shape         Param #     Connected to                     \n",
                        "==================================================================================================\n",
                        " input_3 (InputLayer)           [(None, 128)]        0           []                               \n",
                        "                                                                                                  \n",
                        " input_4 (InputLayer)           [(None, 128)]        0           []                               \n",
                        "                                                                                                  \n",
                        " embedding_2 (Embedding)        (None, 128, 300)     18285000    ['input_3[0][0]']                \n",
                        "                                                                                                  \n",
                        " embedding_3 (Embedding)        (None, 128, 300)     18285000    ['input_4[0][0]']                \n",
                        "                                                                                                  \n",
                        " tf.__operators__.add_1 (TFOpLa  (None, 128, 300)    0           ['embedding_2[0][0]',            \n",
                        " mbda)                                                            'embedding_3[0][0]']            \n",
                        "                                                                                                  \n",
                        " tf.math.subtract_1 (TFOpLambda  (None, 128, 300)    0           ['embedding_2[0][0]',            \n",
                        " )                                                                'embedding_3[0][0]']            \n",
                        "                                                                                                  \n",
                        " tf.math.multiply_1 (TFOpLambda  (None, 128, 300)    0           ['embedding_2[0][0]',            \n",
                        " )                                                                'embedding_3[0][0]']            \n",
                        "                                                                                                  \n",
                        " concatenate_1 (Concatenate)    (None, 128, 900)     0           ['tf.__operators__.add_1[0][0]', \n",
                        "                                                                  'tf.math.subtract_1[0][0]',     \n",
                        "                                                                  'tf.math.multiply_1[0][0]']     \n",
                        "                                                                                                  \n",
                        " lstm_1 (LSTM)                  [(None, 150),        630600      ['concatenate_1[0][0]']          \n",
                        "                                 (None, 150),                                                     \n",
                        "                                 (None, 150)]                                                     \n",
                        "                                                                                                  \n",
                        " dense_1 (Dense)                (None, 2)            302         ['lstm_1[0][2]']                 \n",
                        "                                                                                                  \n",
                        "==================================================================================================\n",
                        "Total params: 37,200,902\n",
                        "Trainable params: 630,902\n",
                        "Non-trainable params: 36,570,000\n",
                        "__________________________________________________________________________________________________\n"
                    ]
                }
            ],
            "source": [
                "model.get_model_summary()"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 24,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "(18000, 128)\n"
                    ]
                }
            ],
            "source": [
                "print(q1_val.shape)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 25,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "Epoch 1/4\n",
                        "9000/9000 [==============================] - 1342s 149ms/step - loss: 0.5511 - accuracy: 0.7175 - val_loss: 0.5226 - val_accuracy: 0.7364\n",
                        "Epoch 2/4\n",
                        "9000/9000 [==============================] - 1363s 151ms/step - loss: 0.4874 - accuracy: 0.7626 - val_loss: 0.5049 - val_accuracy: 0.7547\n",
                        "Epoch 3/4\n",
                        "9000/9000 [==============================] - 1333s 148ms/step - loss: 0.4301 - accuracy: 0.7970 - val_loss: 0.4883 - val_accuracy: 0.7714\n",
                        "Epoch 4/4\n",
                        "9000/9000 [==============================] - 1315s 146ms/step - loss: 0.3721 - accuracy: 0.8288 - val_loss: 0.4709 - val_accuracy: 0.7901\n"
                    ]
                },
                {
                    "data": {
                        "text/plain": [
                            "<keras.callbacks.History at 0x7f1f29bde260>"
                        ]
                    },
                    "execution_count": 25,
                    "metadata": {},
                    "output_type": "execute_result"
                }
            ],
            "source": [
                "model.model.fit([q1_train, q2_train], y_train, epochs = 4, validation_data = ([q1_val, q2_val], y_val), batch_size = 8, validation_batch_size=4, verbose = 1)"
            ]
        },
        {
            "cell_type": "code",
            "execution_count": 26,
            "metadata": {},
            "outputs": [
                {
                    "name": "stdout",
                    "output_type": "stream",
                    "text": [
                        "313/313 [==============================] - 25s 71ms/step\n",
                        "Accuracy:  0.784100015\n",
                        "F1 Score:  0.7357100716392751\n"
                    ]
                },
                {
                    "ename": "",
                    "evalue": "",
                    "output_type": "error",
                    "traceback": []
                }
            ],
            "source": [
                "y_pred = model.predict([q1_test, q2_test])\n",
                "y_pred1d, y_actual1d = [], []\n",
                "for i in range(len(y_test)):\n",
                "    if(y_test[i][0] == 1):\n",
                "        y_actual1d.append(0)\n",
                "    else:\n",
                "        y_actual1d.append(1)\n",
                "\n",
                "for i in range(len(y_pred)):\n",
                "    if(y_pred[i][0] > y_pred[i][1]):\n",
                "        y_pred1d.append(0)\n",
                "    else:\n",
                "        y_pred1d.append(1)\n",
                "\n",
                "from sklearn.metrics import accuracy_score, f1_score\n",
                "print(\"Accuracy: \", accuracy_score(y_actual1d, y_pred1d))\n",
                "print(\"F1 Score: \", f1_score(y_actual1d, y_pred1d))"
            ]
        }
    ],
    "metadata": {
        "kernelspec": {
            "display_name": "Python 3.10.4 64-bit",
            "language": "python",
            "name": "python3"
        },
        "language_info": {
            "codemirror_mode": {
                "name": "ipython",
                "version": 3
            },
            "file_extension": ".py",
            "mimetype": "text/x-python",
            "name": "python",
            "nbconvert_exporter": "python",
            "pygments_lexer": "ipython3",
            "version": "3.10.4"
        },
        "vscode": {
            "interpreter": {
                "hash": "916dbcbb3f70747c44a77c7bcd40155683ae19c65e1c03b4aa3499c5328201f1"
            }
        }
    },
    "nbformat": 4,
    "nbformat_minor": 4
}